build_dataset 해당

In [ ]:
from core.macro_classes.macro_class_dataset import MacroAData

macro_agent = MacroAData()
macro_agent.fetch_data()
macro_agent.add_features()
macro_agent.save_csv()
macro_agent.make_close_price()

In [ ]:
from core.macro_classes.macro_class_dataset import MacroAData
macro_agent = MacroAData()
macro_agent.model_maker()

$$$$   X = macro_sercher(agent, ticker) 역할

In [ ]:
from agents.macro_agent import MacroPredictor
from datetime import datetime

macro_sub = MacroPredictor(
    agent_id='MacroSentiAgent',
    base_date=datetime.today(),
    window=40,
    ticker="NVDA"
)
macro_sub.load_assets()               # 모델, 스케일러 등 불러오기


In [ ]:
macro_sub.fetch_macro_data()          # macro_df 불러오기
X_seq, X_scaled = macro_sub.prepare_features()  # 입력 시퀀스 준비

In [ ]:
print(f"X_seq: {X_seq}")
print(f"X_scaled: {X_scaled}")

$$$$   macro_predictor(X) 역할

In [ ]:
pred_prices, target = macro_sub.m_predictor(X_seq)

In [ ]:
print(f"pred_prices: {pred_prices}")
print(f"target: {target}")

llm 생성 용.. macro_reviewer_draft

In [ ]:
 total_json, opinion = macro_sub.macro_reviewer_draft(X_scaled, pred_prices, target)
 

In [ ]:
print(f"total_json: {total_json}")
print(f"opinion: {opinion}")

reviewer_rebut

In [ ]:
# reviewer_rebut()

[토론 테스트]

In [1]:
from agents.debate_agent import DebateAgent

debate_agent = DebateAgent(1, "NVDA")

C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


In [2]:
debate_agent.run_dataset()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (468, 14, 10), y_seq: (468, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (468 samples, 10 features)
[TRACE B] macro_dataset() start for NVDA


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Data shape: (1304, 90), Columns: 90
[TRACE A] add_features() for self.data:            Open_CL=F  Open_DX-Y.NYB  Open_EURUSD=X    Open_GC=F  Open_HG=F  \
Date                                                                          
2020-01-01        NaN            NaN       1.122083          NaN        NaN   
2020-01-02  61.599998      96.480003       1.121894  1518.099976     2.8165   
2020-01-03  61.180000      96.790001       1.117081  1530.099976     2.7935   
2020-01-06  63.709999      96.900002       1.116246  1580.000000     2.7780   
2020-01-07  62.910000      96.650002       1.119583  1558.300049     2.8010   
...               ...            ...            ...          ...        ...   
2024-12-24  69.559998     108.160004       1.040583  2613.000000     4.0525   
2024-12-25        NaN            NaN       1.040258          NaN        NaN   
2024-12-26  70.199997     108.169998       1.039955  2628.500000     4.0730   
2024-12-27  69.680000     108.0800

[*********************100%***********************]  101 of 101 completed


저장 완료: (1259, 101) rows
macro: 데이터셋 생성> NVDA
[INFO] 병합 후 데이터 shape: (1257, 285)
Epoch 1/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 0.1411 - val_loss: 0.0885
Epoch 2/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.1311 - val_loss: 0.0850
Epoch 3/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.1331 - val_loss: 0.0867
Epoch 4/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.1256 - val_loss: 0.0843
Epoch 5/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.1264 - val_loss: 0.0852
Epoch 6/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.1245 - val_loss: 0.0840
Epoch 7/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.1238 - val_loss: 0.0846
Epoch 8/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - loss: 0.1220 - val_loss: 0.0933
Epoch 9/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step - loss: 0.1243 - val_loss: 0.0970
Epoch 10/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.1215 - val_loss: 0.0883
Epoch 11/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - loss: 0.1233 - 

In [3]:
for agent_id, agent in debate_agent.agents.items():
    scaler = getattr(agent, "scaler", None)
    y_scaler = getattr(scaler, "y_scaler", None)
    print(f"{agent_id}:", type(y_scaler), y_scaler)
    print(agent.ticker)
    # if agent_id =='MacroSentiAgent':
    #     print(agent.macro_df.columns[:20])


TechnicalAgent: <class 'str'> None
NVDA
MacroSentiAgent: <class 'str'> None
NVDA
SentimentalAgent: <class 'str'> None
NVDA


In [4]:
opinions = debate_agent.get_opinion(0, "NVDA")

[*********************100%***********************]  1 of 1 completed


■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
■ TechnicalAgent 모델 없음 → pretrain 수행...
[19:11:28] Pretraining TechnicalAgent
🧠 GRU 모델 생성됨 (input=10, hidden=64, dropout=0.1)
■ TechnicalAgent 모델 새로 생성됨.


C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


  Epoch 005 | Loss: 2.096824
  Epoch 010 | Loss: 2.068167
  Epoch 015 | Loss: 2.127391
  Epoch 020 | Loss: 2.168015
  Epoch 025 | Loss: 2.109215
  Epoch 030 | Loss: 2.088580
  Epoch 035 | Loss: 2.089506
  Epoch 040 | Loss: 2.054442
  Epoch 045 | Loss: 2.057876
  Epoch 050 | Loss: 2.067265
 TechnicalAgent 모델 학습 및 저장 완료: models\NVDA_TechnicalAgent.pt
MacroSentiAgent의 데이터 로드.. macro_sercher
[INFO] 모델 및 스케일러 로드 중...
model_path: models/NVDA_MacroSentiAgent.keras
[OK] 모델 및 스케일러 로드 완료
[INFO] MacroSentimentAgent 데이터 수집 중...
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


...

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\sklearn\linear_model\_least_angle.py:723: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=9.103e-03, with an active set of 2 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\sklearn\linear_model\_least_angle.py:723: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=8.084e-03, with an active set of 3 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\sklearn\linear_model\_least_angle.py:723: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 4 iterations, i.e. alpha=7.949e-03, wi

[Interaction SHAP] Computing correlation among 9 features...

4️⃣ Generating explanation using LLM...

================= pred_prices:{'NVDA': 190.47597791689114} =================

================= LLM Explanation =================
(1) Temporal 분석 결과, 대부분 변수들의 영향력이 최근 시점으로 갈수록 매우 미미하거나 0에 가까워졌으며, 일부 변수인 'Close_HG=F', 'High_USDJPY=X', 'Adj Close_^VIX' 등이 소폭 영향력을 보였다. 이는 최근 시장 변동성이 낮거나 특정 원자재 및 환율 변동이 제한적으로 NVDA 주가에 영향을 미쳤음을 시사한다. (2) Causal 분석에서는 'Open_GC=F'(금 선물 가격)와 'Adj Close_^IXIC'(나스닥 지수 조정 종가)가 음의 인과 효과를 보여 NVDA 주가 하락 요인으로 작용했으며, 반면 'Adj Close_GC=F'는 양의 인과 효과로 긍정적 영향을 주었다. 이는 금 가격 변동과 기술주 지수 간 복합적 상호작용이 존재함을 의미한다. 예를 들어, 금 가격 상승은 안전자산 선호 증가로 기술주에 부정적 영향을 미칠 수 있다. (3) Interaction 분석에서는 'Open_^GSPC'(S&P 500 지수 개장가)와 'Open_^TNX'(10년 만기 국채 수익률), 'High_^DJI'(다우존스 고가), 'Volume_SPY'(S&P 500 ETF 거래량), 'High_HG=F_ret'(구리 선물 고가 수익률) 간 상호작용이 관찰되었다. 이는 주식시장 전반의 지수 및 금리, 원자재 가격 변동이 상호 연계되어 NVDA와 같은 기술주에 복합적 영향을 미친다는 점을 시사한다. (4) 종합 결론으로, NVDA의 예측 종가는 약 190.48로 제시되었으며, 이는 금 가격과 기술주 지수의 복합적 영향 

[*********************100%***********************]  1 of 1 completed


■ SentimentalAgent StockData 생성 완료 (NVDA, USD)
■ SentimentalAgent 모델 없음 → pretrain 수행...
[19:12:38] Pretraining SentimentalAgent
 SentimentalAgent Transformer 생성 완료 (d_model=64, nhead=4, layers=2)
■ SentimentalAgent 모델 새로 생성됨.
  Epoch 005 | Loss: 2.085022
  Epoch 010 | Loss: 2.097316
  Epoch 015 | Loss: 2.049388
  Epoch 020 | Loss: 2.077007
  Epoch 025 | Loss: 2.027503
  Epoch 030 | Loss: 1.993002
  Epoch 035 | Loss: 2.025799
  Epoch 040 | Loss: 2.032858
  Epoch 045 | Loss: 1.988751
  Epoch 050 | Loss: 2.018675
 SentimentalAgent 모델 학습 및 저장 완료: models\NVDA_SentimentalAgent.pt
 Round 0 의견 수집 완료 (3 agents)


In [5]:
print(f"opinions:{opinions}")

opinions:{'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=137.19825342080185, uncertainty=0.1513097584247589, confidence=0.8765008430576077), reason='현재 NVIDIA의 주가는 약 202.49달러로, 우리의 예측 종가는 약 137.20달러로 나타나 약 32% 하락할 것으로 예상됩니다. 이는 최근 가이던스 및 마진 추세가 둔화되고 현금흐름이 예상보다 약화될 가능성을 반영한 결과입니다. 또한, 현재 밸류에이션이 다소 고평가된 상태로 판단되어 조정이 필요하다고 봅니다. 따라서 단기적으로 주가가 하락 압력을 받을 것으로 전망됩니다. 투자자들은 이러한 펀더멘털 변화를 주의 깊게 관찰해야 합니다.'), 'MacroSentiAgent': Opinion(agent_id='MacroSentiAgent', target=Target(next_close=190.476, uncertainty=0.04669049754738808, confidence=21.4176), reason='1. 기술적 분석 측면에서 NVDA의 5일 이동평균선은 약 192.97로 현재 가격 190.48에 근접하며, 10일 이동평균선(약 187.43) 위에 위치해 단기 상승 모멘텀이 존재한다. 거래량은 평균 수준이며, RSI 등 추가 지표는 제공되지 않았으나 이동평균선 간의 골든크로스 가능성이 긍정적 신호로 해석된다. 2. 거시경제 변수로는 금리 지표인 ^TNX(10년물 국채금리)가 4.046%로 다소 높은 수준이며, 인플레이션 압력과 유동성은 직접적 데이터가 없으나, ^FVX(5년물 금리) 3.672%와 ^IRX(단기 금리) 3.729% 수준을 고려하면 금리 상승 압력이 존재한다. 환율은 USDJPY가 152.82로 강세를 보이고 있어 수출기업에 긍정적이나, 금리 상승은 기업 이익에 부담으로 작용할 수 있다. 3. 시장심리 및 정책 방향은 V

리뷰탈 해보기..

In [6]:
debate_agent.get_rebuttal(1)

 Round 1 rebuttals 생성 완료 (6 agents)


[Rebuttal(from_agent_id='TechnicalAgent', to_agent_id='MacroSentiAgent', stance='REBUT', message='상대 에이전트는 5일 및 10일 이동평균선의 위치와 골든크로스 가능성을 근거로 단기 상승 모멘텀을 강조하고 있으나, 이는 매우 보수적인 해석입니다. 현재 주가가 202.49달러임에도 불구하고 137.20달러까지 32% 하락을 예측한 저의 분석은 현 밸류에이션 과대평가와 펀더멘털 약화를 반영한 것으로, 기술적 신호보다 근본적 리스크를 더 중시해야 합니다. 또한, 금리 상승과 정책 불확실성은 기술주에 강한 하락 압력으로 작용할 가능성이 크며, 단기 모멘텀 신호만으로 상승을 낙관하는 것은 과감한 변동성을 간과한 판단입니다. 따라서, 보수적 상승 전망보다는 강한 조정 신호에 무게를 두고 과감한 하락을 예상하는 것이 타당합니다.'),
 Rebuttal(from_agent_id='TechnicalAgent', to_agent_id='SentimentalAgent', stance='REBUT', message='상대 에이전트는 NVDA 주가가 약 63.9% 하락할 것으로 예측하며 매우 보수적인 입장을 취하고 있습니다. 그러나 기술적 분석 전문가로서 현재 주가와 펀더멘털 지표를 고려할 때 32% 하락 전망이 더 합리적이며, 과도한 하락 예측은 시장 과민 반응을 반영한 과장된 신호일 가능성이 큽니다. 또한, 상대의 예측은 구체적인 차트 패턴, 모멘텀, RSI 등의 기술적 신호 분석 없이 단순 여론 데이터 부재를 이유로 크게 하락을 주장하는 점에서 일관성과 대담함이 부족합니다. 따라서 보수적인 과도한 하락 예측은 반박하며, 기술적 신호에 기반한 보다 균형 잡힌 32% 하락 전망을 지지합니다.'),
 Rebuttal(from_agent_id='MacroSentiAgent', to_agent_id='TechnicalAgent', stance='SUPPORT', message='현재 거시경제 국면은 금리 상승 

get_revise 해보기

In [7]:
debate_agent.get_revise(1)

'''
에러 나오는 중..
[TechnicalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
[SentimentalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
'''

TypeError: BaseAgent.reviewer_revise() got an unexpected keyword argument 'revised_target'

결과 도출해보기

In [ ]:
debate_agent.get_ensemble()

통합해서 한번에 실행하기: 기본 라운드는 총 3회. 클래스 정의떄 수정 가능

In [ ]:
#debate_agent.run()